In [ ]:
import open3d as o3d
import numpy as np

from Eval.Mesh import CreatePointcloudFromDir
from Eval.Clouds.Cube import CreateBaseCubePointCloud
from Utils.Viz import VizualiseBaseTargetPointclouds
from Eval.Error import CalculateRMSE
from Utils.Format import MeterToMilimeter, MilimeterToMeter

### Calculate RMSE between Base and Collected Cloud

In [ ]:
# Create the Base Cube Cloud

baseCube = CreateBaseCubePointCloud(
    size = (MilimeterToMeter(1), MilimeterToMeter(1), MilimeterToMeter(1)), 
    resolution = 1000
)

baseCube.paint_uniform_color([0.5, 0.5, 0.5])

In [ ]:
# Current Technique and Data in Review

eval = 'PolyCam'

evalDataDir = f'../data/{eval}/box.ply'

In [ ]:
# Create the Evaluated Cube Cloud

collectedCube = CreatePointcloudFromDir(evalDataDir)
collectedCube.paint_uniform_color([1, 0, 0])

#collectCube.scale(MilimeterToMeter(200), center=collectCube.get_center())

In [ ]:
# Perform ICP Registration between the Base and Collected Cloud

icpRegResults = o3d.registration.registration_icp(
    source = baseCube,
    target = collectedCube,
    max_correspondence_distance = 0.1,
    init = o3d.registration.TransformationEstimationPointToPoint(),
    estimation_method = o3d.registration.TransformationEstimationPointToPoint(),
    criteria = o3d.registration.ICPConvergenceCriteria(max_iteration = 1000)
)

print(icpRegResults)

# Apply the Transform Matrix to align the clouds
baseCube.transform(icpRegResults.transformation)



In [ ]:
# Vizualise Both Clouds Together

VizualiseBaseTargetPointclouds(baseCube, collectedCube)

In [ ]:
# Calculate the RMSE between the Two Clouds

rmse = CalculateRMSE(baseCube, collectedCube)

# Convert RMSE to mm
rmse *= 1000

print(f'RMSE (mm): {rmse}')

### Save and Plot RMSE Data

In [ ]:
from Eval.Plotting import plot_technique_rmse
from Utils.Data import JSON

In [ ]:
# Load the Current RMSE Data from File

jsonFileDir = '../data/techniques-rmse-diagram.json'

rmse_data = JSON(jsonFileDir)

In [ ]:
# Update the RMSE Data

rmse_data.data[eval]["box"].update({
    "rmse": rmse,
    "inlier_rmse": MeterToMilimeter(icpRegResults.inlier_rmse) # Convert to mm (Milimeters)
})

rmse_data.print()

rmse_data.save()

In [ ]:
# Plot the Figure with Current RMSE Data

fig = plot_technique_rmse(rmse_data.data, eval)

fig.show()

In [ ]:
# Save Figure

fig.savefig("../data/techniques-rmse-diagram.png", format="png", dpi=300)